# BERTのファインチューニング入門

In [ ]:
!pip install transformers[torch] ipywidgets datasets accelerate evaluate matplotlib scikit-learn

In [ ]:

from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

from transformers.trainer_utils import set_seed

# 乱数シードを42に固定
set_seed(42)




言語モデルを使ってプログラミング言語の判定を行う。

### データセットを準備する

datasetという、huggingfaceが開発したライブラリを使ってデータの読み込みを行います。

In [ ]:
# https://huggingface.co/datasets/code-search-net/code_search_net

dataset = load_dataset("code-search-net/code_search_net",trust_remote_code=True)

In [ ]:
# データの分布を見る
from collections import Counter
import matplotlib.pyplot as plt


def plot_dataset(dataset):

    # 言語名の出現回数を数える
    try:
        lang_counts = Counter(dataset["train"]["language"])
    except:
        lang_counts = Counter(dataset["language"])




    # 件数の多い順に並べ替え
    labels, values = zip(*sorted(lang_counts.items(), key=lambda x: x[1], reverse=True))

    # グラフを描画
    plt.figure(figsize=(10, 5))
    plt.bar(labels, values)
    plt.ylabel("Number of samples")
    plt.title("Distribution of samples by language (>100 samples only)")
    plt.xticks(rotation=45, ha="right")
    plt.tight_layout()
    plt.show()

plot_dataset(dataset)

- 今回扱うデータは分布が不均衡なデータセット。
- 学習が終わるようにそれぞれの言語のデータセット数を制限

In [ ]:
dataset

### データセットの前処理

In [ ]:
from collections import defaultdict
from datasets import Dataset, DatasetDict
import random

# 元の DatasetDict を使って分割取得
train_dataset = dataset["train"]
test_dataset = dataset["test"]
val_dataset = dataset["validation"]

# DataFrame に変換
df_train = train_dataset.to_pandas()
df_test = test_dataset.to_pandas()
df_val = val_dataset.to_pandas()

def sample_per_language(df, is_validation=False):
    return df.groupby("language").apply(
        lambda x: x.sample(
            n=min(100, len(x)) if not is_validation else min(500, len(x)),
            random_state=42
        )
    ).reset_index(drop=True)


sampled_df_train = sample_per_language(df_train)
sampled_df_test = sample_per_language(df_test)
sampled_df_val = sample_per_language(df_val,is_validation=True)

# Dataset に戻す（index列の削除も忘れずに）
dataset = DatasetDict({
    "train": Dataset.from_pandas(sampled_df_train, preserve_index=False),
    "test": Dataset.from_pandas(sampled_df_test, preserve_index=False),
    "validation": Dataset.from_pandas(sampled_df_val, preserve_index=False),
})





In [ ]:
plot_dataset(dataset)

In [ ]:
# 関数のトークナイズ処理を定義
def tokenize_function(example):
    tokenized_example = tokenizer(example["whole_func_string"], max_length=512)
    # ラベルとして言語ID（数値）を追加
    tokenized_example["labels"] = example["language"]
    return tokenized_example

# 事前学習済みBERTモデルのトークナイザーを読み込み
model_name = "google-bert/bert-large-cased"
tokenizer  = AutoTokenizer.from_pretrained(model_name)    

# 元の言語ラベル（文字列）を保存する列を追加（後で逆変換のために利用できる）
dataset = dataset.map(lambda x: {"language_str": x["language"]})

# 言語列をカテゴリとしてエンコード（数値に変換）
dataset = dataset.class_encode_column("language")

# 各データをトークナイズし、ラベル情報を付加
tokenize_dataset = dataset.map(tokenize_function)


## モデルの読み込みと設定

In [ ]:
# dataset["train"]
#　目的変数をlanguageとして分類タスクを行う

In [ ]:
#dataset["train"][0]

In [ ]:
# google-bert/bert-large-cased https://huggingface.co/docs/transformers/main/en/model_doc/auto#natural-language-processing

model = AutoModelForSequenceClassification.from_pretrained(model_name,
                                                           num_labels=len(set(dataset["train"]['language'])),
                                                           device_map="cuda:0"
                                                           )

In [ ]:
model.classifier

In [ ]:
import numpy as np

def compute_accuracy(
    eval_pred: tuple[np.ndarray, np.ndarray]
) -> dict[str, float]:
    """予測ラベルと正解ラベルから正解率を計算"""
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)

    # (predictions == labels)        → [True, False, True]
    # (predictions == labels).mean() → (1 + 0 + 1) / 3 


    return {"accuracy": (predictions == labels).mean()}



In [ ]:
# モデルの学習に関する設定をまとめるオブジェクト
training_args = TrainingArguments(
    output_dir="bert-classification-language",  # 学習済みモデルの保存先ディレクトリ
    learning_rate=2e-5,                         # 学習率（小さすぎても大きすぎても学習が不安定になるため適切に設定）
    per_device_train_batch_size=16,            # 1デバイスあたりの学習時バッチサイズ
    per_device_eval_batch_size=16,             # 1デバイスあたりの評価時バッチサイズ
    eval_strategy="epoch",                     # 各エポック終了時に評価を実施
    logging_strategy="epoch",                  # 各エポック終了時にログを記録
    num_train_epochs=3,                        # 学習のエポック数
    weight_decay=0.01,                         # 過重みの減衰
    report_to="none"                           # ロギングツール（例: TensorBoard）への出力を行わない
)

In [ ]:
from transformers import DataCollatorWithPadding


# トークナイズされた入力データに対して、バッチごとに自動でパディングを行う
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)




In [ ]:
trainer = Trainer(
    model=model,
    train_dataset=tokenize_dataset["train"],
    eval_dataset=tokenize_dataset["test"],
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_accuracy,
)
trainer.train()

In [ ]:
sample = dataset["validation"][1]
inputs = tokenizer(sample["whole_func_string"], return_tensors="pt").to("cuda")
pred   = model(**inputs).logits.argmax(-1).item()

In [ ]:
import torch

sample = dataset["validation"][1]
inputs = tokenizer(sample["whole_func_string"], return_tensors="pt").to("cuda")
with torch.no_grad():
    pred = model(**inputs).logits.argmax(-1).item()

is_correct = pred == sample["language"]
print(f"予測ラベル: {pred}, 正解ラベル: {sample['language']}, 一致しているか: {is_correct}")

In [ ]:
# 正解数のカウント
correct = 0

# 検証データのサンプル数を取得
total = len(dataset["validation"])

# 検証データの全サンプルに対してループ
for i in range(total):
    sample = dataset["validation"][i]

    # 入力テキストをトークナイズ
    inputs = tokenizer(
        sample["whole_func_string"],
        return_tensors="pt",       # PyTorch形式のテンソルに変換
        truncation=True,           # 最大長を超える部分は切り捨て
        padding=True               # 短い文はパディングで調整
    ).to("cuda")                   # GPU上で処理

    # 推論モード
    with torch.no_grad():
        # モデルに入力して予測ラベルを取得
        pred = model(**inputs).logits.argmax(-1).item()

    # 予測が正解ラベルと一致していればカウントを増やす
    if pred == sample["language"]:
        correct += 1

# 全体に対する正解率を計算
accuracy = correct / total

# 結果を出力
print(f"正解数: {correct}/{total}, 正解率: {accuracy:.2%}")


In [ ]:
SAVE_DIR = "./language-bert-model"
trainer.save_model(SAVE_DIR)                # モデルの保存
tokenizer.save_pretrained(SAVE_DIR)              # トークナイザも忘れずに